# Requirements

In [40]:
import sys

# Problem setting

Python as a type `list` that works peerfectly well, and the implementation is well-optimized.  However, Python lists are ephemeral, not persistent, i.e., if you, e.g., append an element to a list, the original list no longer exists. In other words, the `append` method has side effects.

While this is fine (and likely expected) for an imperative programming language, this is not acceptable in functional programming where you expect functions to be side effect-free.

For persistent lists, some operations will have to copy part of the data structure, not the values stored in that data structure, but pats of the structure itself.  This will of course have a performance impact.  On the upside, this implies that persistent data structures can be used without hassle in parallel programs.

Data structures can be made persistent, i.e., they support multiple versions.  Here this is illustrated by the implementation of a persistent list.

# Data structure

A list will be represented using `tuple` and `None`.  The empty list is represented by `None`, a list with a single element the 2-tuple `('a', None)`, while a list with two elements is a 2-tuple that has the first element of the list as its first element, the second element of the list as its second, under the form of a tuple: `('a', ('b', None))`

More generally, a list is represented by a tuple, the first element is the head of the list, the second the tail, i.e., all other elements in a list, so for a list with four elements:
```
('a', ('b', ('c', ('d', None))))
```

Since a `tuple` in Python is immutable, persistence is already partly guaranteed since once created, a tuple can not be modified.

# Basic operations

You need three basic functions for persistence lists, and those will let you implement all other list operations:
* `cons(x, xs)`: construct a list by prepending the element `x` to the list `xs`;
* `head(xs)`: return the head of the list `xs`, i.e., its first element;
* `tail(xs)`: return the tail of the list, i.e., a list containing all elements, except the first.

Applying `head` or `tail` to an empty list should raise an exception, `ValueError` is appropriate.  For convenience, we also implement `is_empty(xs)` that checks whether a list is empty, as well as a function `empty()` that returns an empty list.

In [75]:
def empty():
    '''Create an empty list
    
    Returns
    -------
    None
        empty list
    '''
    return None

In [73]:
def is_empty(xs):
    '''Returns True if the list is empty, false otherwise
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
        list to check for emptiness
        
    Returns
    -------
    bool
        True if the list is empty, false otherwise
    '''
    return xs is None

In [77]:
def cons(x, xs):
    '''Construct a list with x as head and xs as tail
    
    Parameters
    ----------
    x: Any
        value to be the first element in the new list
    xs: tuple[Any, tuple | None]
        list that is the tail of the new list
    
    Returns
    -------
    tuple[Any, tuple | None]
        new list
    '''
    return (x, xs)

In [78]:
def head(xs):
    '''Return the head of the string
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
        list to get the first element of
        
    Returns
    -------
    Any
        value of the first element in the list
        
    Raises
    ------
    ValueError
        if the list is empty, i.e., xs == None
    '''
    if is_empty(xs):
        raise ValueError('empty list')
    return xs[0]

In [79]:
def tail(xs):
    '''Return the tail of the string
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
        list to get the tail of as a list
        
    Returns
    -------
    tuple[Any, tuple | None] | None
        tail of the list, note that the tail of a single-element list is the empty list,
        so None
        
    Raises
    ------
    ValueError
        if the list is empty, i.e., xs == None
    '''

    if is_empty(xs):
        raise ValueError('empty list')
    return xs[1]

To test this, you can construct list of lengths up to some value.  However, lets first verify that the empty list is empty.

In [86]:
is_empty(empty())

True

In [83]:
lists = [empty()]
for i in range(4, 0, -1):
    lists.append(cons(i, lists[-1]))
for l in lists:
    if is_empty(l):
        print(f'{l} empty')
    else:
        print(f'{l}\n\thead = {head(l)}\n\ttail = {tail(l)}')

None empty
(4, None)
	head = 4
	tail = None
(3, (4, None))
	head = 3
	tail = (4, None)
(2, (3, (4, None)))
	head = 2
	tail = (3, (4, None))
(1, (2, (3, (4, None))))
	head = 1
	tail = (2, (3, (4, None)))


Check that the expected exceptions are raised when `head` and `tail` are applied to empty lists.

In [84]:
try:
    head(empty())
except ValueError as e:
    print(f'Expected exception raised: {e}')

Expected exception raised: empty list


In [85]:
try:
    tail(empty())
except ValueError as e:
    print(f'Expected exception raised: {e}')

Expected exception raised: empty list


Note that although the elements were "inserted" in the previous list, those lists were not modified, hence they are persistent, as required.

# None basic operations

Using `empty`, `is_empty`, `cons`, `head` and `tail`, we can define every list operation we like.  We define a few lists that we can use to test the implementations.

In [89]:
l1 = cons(1, cons(2, cons(3, cons(4, empty()))))

In [90]:
l2 = cons('a', cons('b', cons('c', empty())))

## Iterator

Let's first implement an iterator over the elements of the list since this makes some operations more convenient.

In [88]:
def elements(xs):
    '''Iterate over the elements of a list
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
    
    Returns
    -------
    Any | None
        elements of the list, None when all elements were returned
    '''
    if not is_empty(xs):
        yield xs[0]
        yield from elements(tail(xs))

In [91]:
for element in elements(l1):
    print(element)

1
2
3
4


## Flatten

This is a function that can be used to simplify testing and visualization, it will convert a persistent list of $N$ elements into an $N$-tuple that has the same elements in the same order.

In [92]:
def flatten(xs):
    '''Flatten a list into a tuple
    
    Parameeters
    -----------
    xs: tuple[Any, tuple | None] | None
        list to flatten
        
    Returns
    -------
    tuple[Any] | None
        tuple representing the list or an empty list if the original list was empty
    '''
    if is_empty(xs):
        return xs
    return tuple(x for x in elements(xs))

In [93]:
flatten(l1)

(1, 2, 3, 4)

In [21]:
def element_at(xs, i):
    if xs is None:
        raise ValueError('empty list')
    if i == 0:
        return head(xs)
    return element_at(tail(xs), i - 1)

In [22]:
element_at(l, 3)

4

In [26]:
def contains(xs, x):
    if xs is None:
        return False
    if x == head(xs):
        return True
    return contains(tail(xs), x)

In [27]:
contains(l, 2)

True

In [28]:
contains(l, 5)

False

In [29]:
def length(xs):
    if xs is None:
        return 0
    return 1 + length(tail(xs))

In [30]:
length(l)

4

In [32]:
def insert_at(xs, x, i):
    if i == 0:
        return cons(x, xs)
    if xs is None or i < 0:
        raise ValueError('index out of bounds')
    return cons(head(xs), insert_at(tail(xs), x, i - 1))

In [33]:
l1 = insert_at(l, 13, 2)

In [34]:
l1

(1, (2, (13, (3, (4, None)))))

In [35]:
l

(1, (2, (3, (4, None))))

In [37]:
insert_at(l, 13, 4)

(1, (2, (3, (4, (13, None)))))

In [42]:
try:
    insert_at(l, 13, 5)
except ValueError as e:
    print(f'error: {str(e)}', file=sys.stderr)

error: index out of bounds


In [50]:
def remove_at(xs, i):
    if xs is None:
        raise ValueError('index out of bounds')
    if i == 0:
        return tail(xs)
    return cons(head(xs), remove_at(tail(xs), i - 1))

In [44]:
remove_at(l, 1)

(1, (3, (4, None)))

In [46]:
remove_at(l, 0)

(2, (3, (4, None)))

In [48]:
remove_at(l, 3)

(1, (2, (3, None)))

In [51]:
try:
    remove_at(l, 4)
except ValueError as e:
    print(f'error: {str(e)}', file=sys.stderr)

error: index out of bounds


In [53]:
def concat(xs, ys):
    if xs is None:
        return ys
    return cons(head(xs), concat(tail(xs), ys))

In [59]:
l2 = cons('a', cons('b', cons('c')))

In [60]:
concat(l, l2)

(1, (2, (3, (4, ('a', ('b', ('c', None)))))))

In [61]:
l

(1, (2, (3, (4, None))))

In [64]:
def reverse(xs):
    if xs is None:
        return None
    return concat(reverse(tail(xs)), cons(head(xs)))

In [65]:
reverse(l)

(4, (3, (2, (1, None))))